# Data Prep

The goal of this notebook is to prep the data for the alignment task.  This includes computing audio features and generating a query list.

In [ ]:
import numpy as np
import librosa as lb
import os
import os.path
from pathlib import Path
import multiprocessing

In [ ]:
DATA_ROOT = Path('../../data/')
AUDIO_ROOT = DATA_ROOT / Path('Chopin_Mazurkas/wav_22050_mono')
FEATURES_ROOT = Path('features') # directory which will store chroma features
TRAIN_DATASET = "toy"
%store TRAIN_DATASET
%store DATA_ROOT

train_files = Path('cfg_files/filelist.' + TRAIN_DATASET + '.txt')
test_files = Path('cfg_files/filelist.test.txt')

In [ ]:
if not os.path.exists(FEATURES_ROOT):
    os.mkdir(FEATURES_ROOT)

### Compute features on clean audio

First we compute features on the audio.

In [ ]:
def compute_chroma_single(infile, outfile, sr = 22050, hop_length=512):
    y, sr = lb.core.load(infile, sr = sr)
    #F = lb.feature.chroma_cens(y, sr=sr, hop_length=hop_length)
    F = lb.feature.chroma_cqt(y, sr=sr, hop_length=hop_length, norm=2) # TODO: why do chroma features need a window?
    np.save(outfile, F)
    return

In [ ]:
def compute_chroma_batch(filelist, outdir, n_cores):

    # prep inputs for parallelization
    inputs = []
    with open(filelist, 'r') as f:
        for line in f:
            relpath = line.strip()
            reldir, fileid = os.path.split(relpath)
            featdir = outdir / reldir
            featdir.mkdir(parents=True, exist_ok=True)
            featfile = (featdir / fileid).with_suffix('.npy')
            audiofile = (AUDIO_ROOT / relpath).with_suffix('.wav')
            if os.path.exists(featfile):
                print(f"Skipping {featfile}")
            else:
                inputs.append((audiofile, featfile))

    # process files in parallel
    pool = multiprocessing.Pool(processes = n_cores)
    pool.starmap(compute_chroma_single, inputs)
    
    return

In [ ]:
FEATS_CLEAN_DIR = FEATURES_ROOT / 'clean' # TODO: why clean? what do we do to the others?
compute_chroma_batch(train_files, FEATS_CLEAN_DIR, 24)
compute_chroma_batch(test_files, FEATS_CLEAN_DIR, 24)

### Generate query list

Here we generate a file containing each pair of files to be aligned.

The lines of this outfile have the form
```
piece_name/recording_name_1 piece_name/recording_name_2
```

In [ ]:
def generate_query_list(filelist, outfile):
    
    # group files by piece
    d = {}
    with open(filelist, 'r') as f:
        for line in f:
            parts = line.strip().split('/')
            assert len(parts) == 2
            piece, fileid = parts
            if piece not in d:
                d[piece] = []
            d[piece].append(fileid)
            
    # print out all pairings
    with open(outfile, 'w') as fout:
        for piece in d:
            num_recordings = len(d[piece])
            for i in range(num_recordings):
                fileid1 = d[piece][i]
                for j in range(i+1, num_recordings):
                    fileid2 = d[piece][j]
                    line = f'{piece}/{fileid1} {piece}/{fileid2}\n'
                    fout.write(line)
                    
    return

In [ ]:
train_queries = 'cfg_files/query.' + TRAIN_DATASET + '.list'
test_queries = 'cfg_files/query.test.list'
generate_query_list(train_files, train_queries)
generate_query_list(test_files, test_queries)